In [7]:
# interpreter pytorch_py3.7

import pandas as pd
import re
import numpy as np
import os
# from read_file import *

def newDic(index, name, first, second, third, fourth, sentence):
    return {
        'file_index': index,
        'class':name,
        'first':first,
        'second':second,
        'third':third,
        'fourth': fourth,
        'sentence':sentence,
    }


if __name__ == '__main__':

    #1. 文件读取
    root = '../documents/'
    files = os.listdir(root)
    info = []
    for file in files:
        if '.txt' in file:
            f = open(root + file, 'r', encoding='utf-8')
            text = f.read()
            info.append(text)
            f.close()
        elif '.doc' in file:
            pass
        elif '.ppt' in file or '.pptx' in file:
            pass
        elif '.pdf' in file and file.replace('.pdf', '.txt') not in files:
#             text = pdf2text(root, file)
            pass
        elif '.xlsx' in file or '.xls' in file:
            pass
        else: # 想不到先
            pass

#     exit()
#     file = pd.read_excel('物流数据内容（中诺思提供）.xlsx', sheet_name=None)
#     data = file['Sheet2']


In [244]:
PATTERN_TITLE_LIST = {
    '[一二三四五六七八九十]+\s*、|附\s*[录表][：\:\d]?': 0, 
    '（\s*[一二三四五六七八九十]+\s*）|\(\s*[一二三四五六七八九十]+\s*\)': 1,
    '\d\s*[\.\．、]': 2,
    '\d{1,2}\s?[\.．]\s?\d{1,2}':3,
    '\(?\d\)|（?\d）':4,
    
    '表\d+\s|项\s*目\s*[一二三四五六七八九十0-9]\s*、?|模\s*块?\s*[一二三四五六七八九十0-9]\s*、?|[\u4e00-\u9fa5_a-zA-Z0-9]{2,6}\s*[\:：]':999,
}

PATTERN_LABEL_NUMBER = r'[^一二三四五六七八九十0-9]+'

def stuctureIdentifier(text):
    def next_label(label): pass
    
    def next_number(label):
        if label in '一 二 三 四 五 六 七 八 九 十 十一 十二 十三 十四 十五'.split(' '):
            tmp = '一 二 三 四 五 六 七 八 九 十 十一 十二 十三 十四 十五'.split(' ')
            try: return tmp[tmp.index(label)+1]
            except: return '十六'
        else:
            try: return str(int(label)+1)
            except: pass #TODO: 没想到还有什么情况
        return 'unnamed'
    
    
    for title_pattern in PATTERN_TITLE_LIST:
#         print('>> ', title_pattern)
        # 先全标
        pattern = r"([\n\s\t]+)("+title_pattern+ r")(\s*[\u4e00-\u9fa5_a-zA-Z])"
        text = re.sub(pattern, r'\1<title' + str(PATTERN_TITLE_LIST[title_pattern]) + r'>\2</title' + str(PATTERN_TITLE_LIST[title_pattern]) + r'>\3', text)

        
    for title_pattern in PATTERN_TITLE_LIST:
        finded = re.finditer(f'<title{PATTERN_TITLE_LIST[title_pattern]}>(.+?)</title{PATTERN_TITLE_LIST[title_pattern]}>(.+?)\n', text)
        #再过滤掉层级不对的label <- 并不需要提前分句
        for it in finded:
            print(text[it.start():it.end()])
            above_text = text[: it.start()]
            current_class = re.findall('<title(\d+)>', text[it.start():it.end()])[0]
            current = re.findall('<title\d+>(.*?)</title\d+>', text[it.start():it.end()])[0]
            current_tag = re.sub(PATTERN_LABEL_NUMBER, '', current)
            if len(current_tag) == 0: continue
            if (str(current_tag) in ['1', '一'] and not ('.' in current or '．' in current)) or\
                (list(str(current_tag))[-1] in ['1', '一'] and ('.' in current or '．' in current)): 
                if len(re.findall('<title(\d+)>',above_text)) > 0:
                    print('test', text[it.start():it.end()],re.findall('<title\d+>.*?\s',above_text)[-1], int(re.findall('<title(\d+)>',above_text)[-1]), int(current_class),
                         int(re.findall('<title(\d+)>',above_text)[-1]) < int(current_class))
                    if int(re.findall('<title(\d+)>',above_text)[-1]) <= int(current_class): continue
#                     elif int(re.findall('<title(\d+)>',above_text)[-1]) < int(current_class) and \
#                             list(str(re.sub(PATTERN_LABEL_NUMBER, '', re.findall('<title\d+>(.*?)</title',above_text)[-1])))[-1] in ['1', '一']: 
#                         print('cas1')
#                         continue
                    else:
                        tmp = list(text)
                        tmp[it.start():it.end()] = re.sub('title','delet',text[it.start():it.end()])
                        text = ''.join(tmp)
                        continue

                
                
            above_finded = re.findall(f'<title{str(current_class)}>(.*?)</title{str(current_class)}>', above_text)
            if len(above_finded) > 0:
                last_tag = re.sub(PATTERN_LABEL_NUMBER, '', above_finded[-1])
                if current_tag==next_number(last_tag): 
                    last_delete = re.findall(f'<delet{str(current_class)}>(.*?)</delet{str(current_class)}>', above_text)
                    if len(last_delete) > 0:
                        if current_tag==next_number(re.sub(PATTERN_LABEL_NUMBER, '', last_delete[-1])):
                            tmp = list(text)
                            tmp[it.start():it.end()] = re.sub('title','delet',text[it.start():it.end()])
                            text = ''.join(tmp)
                            continue
                        else: continue
                    else: continue

                        
                #判断next同级标签
                below_finded = re.findall(f'<title{str(current_class)}>(.*?)</title{str(current_class)}>', text[it.end():])
                if len(below_finded) > 0:
                    if (below_finded[0] == next_number(current_tag)) and (current_tag == last_tag): continue
                        
                #判断next标签是否为一个新的子标签
                below_finded = re.findall(f'<title(\d+)>(.*?)</title\d+>', text[it.end():])
                if len(below_finded) > 0:
                    next_class = re.sub(PATTERN_LABEL_NUMBER, '', below_finded[0][1])
#                     print(below_finded[0], next_class)
                    try:
                        if below_finded[0][0] != current_class and (str(next_class) in ['1', '一'] or\
                            (list(str(next_class))[-1] in ['1', '一'] and ('.' in below_finded[0][1] or '．' in below_finded[0][1]))): continue
                    except: pass
                tmp = list(text)
                tmp[it.start():it.end()] = re.sub('title','delet',text[it.start():it.end()])
                text = ''.join(tmp)
                print('update to :', text[it.start():it.end()])

            print('next')
#             print(TITLE_LIST[title_pattern],it.group(), it.start(), it.end(), text[it.start():it.end()])
    return re.sub('</?delet\d+>', '', text)

In [245]:
print(stuctureIdentifier(info[0]))

<title0>一 、</title0>课程性质

next
<title0>二、</title0> 职业活动

<title0>三、</title0>  课程目标

<title0>四、</title0>课程内容

<title0>一、</title0>会计法律

test <title0>一、</title0>会计法律
 <title999>模块一、</title999>会计法律制度的构成
 999 0 False
<title0>二、</title0>会计行政法规

update to : <delet0>二、</delet0>会计行政法规

next
<title0>三、</title0>国家统一的会计制度

update to : <delet0>三、</delet0>国家统一的会计制度

next
<title0>一、</title0>会计工作的行政管理

test <title0>一、</title0>会计工作的行政管理
 <title999>模块二、</title999>会计工作管理体制
 999 0 False
<title0>二、</title0>会计工作的自律管理

<title0>三、</title0>单位会计工作管理

<title0>一、</title0>总体要求

test <title0>一、</title0>总体要求
 <title999>模块三、</title999>会计法律核算
 999 0 False
<title0>二、</title0>会计凭证

update to : <delet0>二、</delet0>会计凭证

next
<title0>三、</title0>会计账簿

update to : <delet0>三、</delet0>会计账簿

next
<title0>四、</title0>财务会计报告

<title0>五、</title0>会计档案

update to : <delet0>五、</delet0>会计档案

next
<title0>六、</title0>其他

update to : <delet0>六、</delet0>其他

next
<title0>一、</title0>单位内部会计监督

test <title0>一、</title0>单位内部会计监督
 <title999>模块四、<

update to : <delet4>(8)</delet4>把握商品管理方法	

next
<title999>项目一  </title999>会计法律制度(22课时)

test <title999>项目一  </title999>会计法律制度(22课时)
 <title2>3.</title2>项目设计
 2 999 True
<title999>模块一、</title999>会计法律制度的构成

test <title999>模块一、</title999>会计法律制度的构成
 <title999>项目一  999 999 False
<title999>模块二、</title999>会计工作管理体制

<title999>模块三、</title999>会计法律核算

<title999>模块四、</title999>会计监督

<title999>模块五、</title999>会计机构和会计人员

<title999>模块六、</title999>会计法律责任

<title999>项目二、</title999>支付结算法律制度22课时

update to : <delet999>项目二、</delet999>支付结算法律制度22课时

next
<title999>模块一</title999>概述

test <title999>模块一</title999>概述
 <title999>模块六、</title999>会计法律责任
 999 999 False
<title999>模块二   </title999>现金管理

<title999>模块三 </title999>银行结算账户

<title999>模块四 </title999>票据结算方式

update to : <delet999>模块四 </delet999>票据结算方式

next
<title999>项目三</title999>税收法律制度(30课时)

<title999>模块二、</title999>主要税种

update to : <delet999>模块二、</delet999>主要税种

next
<title999>模块三  </title999>税收征管(12课时)

update to : <delet999>模块三  </delet999>税收征管(12课时)



In [207]:
data = pd.read_excel('../物流数据内容（中诺思提供）.xlsx', sheet_name=['Sheet2'])
data = data['Sheet2']
data

,文件内容,课程名称,专业名称,课程类型
0,\n物流管理专业\n\n\n \n “物流市场营销”\n 课...,物流市场营销,物流管理,专业核心课程
1,《Excel应用》课程标准\n二、课程性质\n 《Excel实用教程》是物流专业的...,Excel应用,物流管理,专业基础课程
2,《交通工程》课程标准\n二、课程性质\n 《交通工程学》是物流专业的一门专业基础课程，理...,交通工程,物流管理,专业基础课程
3,\n物流管理专业\n\n\n \n“仓储与配送管理实训”\n实训指导书\...,仓储与配送管理实训,物流管理,专业拓展课程
4,《仓储与配送管理》课程标准\n 一、课程标识\n 课程代码：4031308\n ...,仓储与配送管理,物流管理,专业核心课程
...,...,...,...,...
67,《采购管理实训》课程标准\n 二、课程概述\n （一）课程性质\n 《采购管理》...,采购管理实训,物流管理,专业拓展课程
68,《国际货代客服》课程标准\n目 录\n一、课程定位与设计思路\t1\n（一）课程定位\t1...,国际货代客服,物流管理,专业基础课程
69,《物流信息技术与应用》课程标准\n\t\t\t\n\t\t\t\n\t\t\t\n\t\t\...,物流信息技术与应用,物流管理,专业核心课程
70,《 安全管理学 》 课程标准\n课程代码： \n建议课时数： 64\n适用专业：安全工程\n...,安全管理学,安全工程,专业基础课程


In [222]:
data['clean'] = ''
for index, row in data.iterrows():
    data['clean'][index] = stuctureIdentifier(data['文件内容'][index])
data.to_excel('text1513.xlsx')

PermissionError: [Errno 13] Permission denied: 'text1513.xlsx'